# 📙 Carga de Datos a PostgreSQL
Este notebook inserta los datos limpios almacenados en archivos `.csv` a una base de datos PostgreSQL.

## 📦 1. Importar librerías necesarias y secretos de conexión
Usamos `psycopg2` para conectar con PostgreSQL y `pandas` para manejar los datos.

In [ ]:
import os
import pandas as pd
import psycopg2
import hidden1  # Asegúrate de que este archivo existe con tu diccionario de conexión

## 🔐 2. Conectar a la base de datos
Se extraen las credenciales desde `hidden1.py` y se establece la conexión.

In [ ]:
secrets = hidden1.secrets()
conn = psycopg2.connect(
    host = secrets['host'],
    port = secrets['port'],
    database = secrets['database'],
    user = secrets['user'],
    password = secrets['pass']
)
cur = conn.cursor()

## 📁 3. Cargar y recorrer archivos `.csv` limpios
Insertamos los datos en las tablas correspondientes, identificando si el archivo es de RUL o de sensores.

In [ ]:
clean_dir = 'data/cleaned'

for file in os.listdir(clean_dir):
    if not file.endswith('.csv'):
        continue

    path = os.path.join(clean_dir, file)
    table = file.replace('.csv', '')
    table_name = f'{table}_raw'

    if table.lower().startswith('rul'):
        df = pd.read_csv(path, header=0, names=['rul'])
        for value in df['rul']:
            cur.execute(f'INSERT INTO "{table}" (rul) VALUES (%s);', (int(value),))
        print(f'🗃️ Insertados {len(df)} valores en {table}')

    else:
        columns = ['unit_number', 'time_in_cycles'] + [f'operational_setting{i}' for i in range(1, 4)] + [f'sensor_measurement{j}' for j in range(1, 22)]
        df = pd.read_csv(path, header=0, names=columns)
        placeholders = ', '.join(['%s'] * len(columns))
        cols = ', '.join(columns)

        for row in df.itertuples(index=False, name=None):
            cur.execute(f'INSERT INTO "{table_name}" ({cols}) VALUES ({placeholders});', row)
        print(f'🗃️ Insertados {len(df)} valores en {table_name}')

## ✅ 4. Confirmar y cerrar conexión
Guardamos los cambios y cerramos la conexión de manera segura.

In [ ]:
conn.commit()
cur.close()
conn.close()
print("🚀 Todos los datos fueron cargados exitosamente.")